In [4]:
import torch
from torch import nn

# 배치 정규화

- **배치 정규화(Batch Normalization)**란 **내부 공변량 변화(Internal Covariate Shift)**를 줄여 과대적합을 방지하는 기술이다.
- 각 계층은 배치 단위의 데이터로 인해 계속 변화되는 입력 분포를 학습해야 하기 때문에 인공 신경망의 성능과 안정성이 낮아져 학습 속도가 느려진다. 내부 공변량 변화란 이렇게 계층마다 입력 분포가 변경되는 현상을 의미한다.

In [2]:
x = torch.FloatTensor(
    [
        [-0.6577, -0.5797, 0.6360],
        [0.7392, 0.2145, 1.523],
        [0.2432, 0.5662, 0.322]
    ]
)

배치 정규화는 feature마다 따로 계산된다.

In [5]:
print(nn.BatchNorm1d(num_features=3)(x))

tensor([[-1.3246, -1.3492, -0.3756],
        [ 1.0912,  0.3077,  1.3685],
        [ 0.2334,  1.0415, -0.9930]], grad_fn=<NativeBatchNormBackward0>)


# 가중치 초기화

- 상수 초기화: 모든 가중치를 매우 작은 양의 상수값으로 동일하게 할당 -> 대칭 파괴 현상으로 모델을 학습하기 어렵거나 불가능하게 만든다.
- 무작위 초기화: 초기 가중치를 무작위나 특정 분포 형태로 초기화 하는 것으로 random, uniform distribution, normal distribution, truncated normal distribution, sparse normal distribution 등이 있다.

- 자주 사용하는 초기화 방법으로 Xavier와 He가 있다. Xavier는 이전 계층과 다음 계층의 노드 수를 고려하는 방법으로 입력 데이터의 분산이 출력 데이터에서 유지되도록 가중치를 초기화해 시그모이드나 하이퍼볼릭 탄젠트를 활성화 함수로 사용하는 네트워크에서 효과적이다.  
- He는 현재 계층의 입력 뉴런수만 고려한 방법으로 각 노드의 출력 분산이 입력 분산과 동일하게 만들어 ReLU를 사용하는 네트워크에서 효과적이다.

## 가중치 초기화 함수 (1)

In [6]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Linear(1, 2),
        nn.Sigmoid(),
    )
    self.fc = nn.Linear(2, 1)
    self._init_weights()

  def _init_weights(self): # Protected Method
    nn.init.xavier_uniform_(self.layer[0].weight)
    self.layer[0].bias.data.fill_(0.01)

    nn.init.xavier_uniform_(self.fc.weight)
    self.fc.bias.data.fill_(0.01)

model = Net()

## 가중치 초기화 함수 (2)

모델이 커지고 구조가 복잡한 경우 초기화 메서드를 모듈화해 적용한다

In [9]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Linear(1, 2),
        nn.Sigmoid(),
    )
    self.fc = nn.Linear(2, 1)
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      nn.init.xavier_normal_(module.weight)
      nn.init.constant_(module.bias, 0.01)
      print(f"Apply : {module}")

model = Net()

Apply : Linear(in_features=1, out_features=2, bias=True)
Apply : Linear(in_features=2, out_features=1, bias=True)


# 정칙화

## L1

In [ ]:
for x, y in train_dataloader:
  x = x.to(device)
  y = y.to(device)

  output = model(x)

  _lambda = 0.5
  l1_loss = sum(p.abs().sum() for p in model.parameters())

  loss = criterion(output, y) + _lambda * l1_loss

## L2

In [ ]:
for x, y in train_dataloader:
  x = x.to(device)
  y = y.to(device)

  output = model(x)

  _lambda = 0.5
  l2_loss = sum(p.pow(2.0).sum() for p in model.parameters())

  loss = criterion(output, y) + _lambda * l2_loss

## 가중치 감쇠

파이토치에서 가중치 감쇠와 L2 정칙화는 동일하다

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)

## 드롭아웃

In [ ]:
class Net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = torch.nn.Linear(10, 10)
    self.dropout = torch.nn.Dropout(p=0.5)
    self.layer2 = torch.nn.Linear(10, 10)

  def forward(self, x):
    x = self.layer1(x)
    x = self.dropout(x)
    x = self.layer2(x)
    return x

## 그레디언트 클리핑

In [ ]:
grad_norm = torch.nn.utils.clip_grad_norm_(
    parameters,
    max_norm,
    norm_type=2.0,
)

In [ ]:
for x, y in train_dataloader:
  x = x.to(device)
  y = y.to(device)

  output = model(x)
  loss = criterion(output, y)

  optimizer.zero_grad()
  loss.backward()

  torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

  optimizer.step()